In [5]:
"""
In this notebook, I will be discussing how to extract important
#information from the json file that was created after running 
#predicate.py. All the information should be in the json file called 
#data.json. 
"""
import json
import numpy as np
import os
from PIL import Image
import numpy as np
import Imath
from matplotlib.pyplot import imshow

#BEFORE RUNNING, YOU MAY NEED TO DOWNLOAD OPENEXR.
#https://anaconda.org/conda-forge/openexr-python

In [ ]:
#BE ABLE TO SEE THE SCENE AND THE JSON FILE FOR EACH SCENE

In [3]:
import OpenEXR as exr
def extract_rgb_depth(filename):
    exrfile = exr.InputFile(filename)
    header = exrfile.header()
    dw = header['dataWindow']
    isize = (dw.max.y - dw.min.y + 1, dw.max.x - dw.min.x + 1)
    channelData = dict()
    for c in header['channels']:
        C = exrfile.channel(c, Imath.PixelType(Imath.PixelType.FLOAT))
        C = np.frombuffer(C, dtype=np.float32)
        C = np.reshape(C, isize)
        
        channelData[c] = C
    depth = channelData['Z']
    colorChannels = ['R', 'G', 'B', 'A'] if 'A' in header['channels'] else ['R', 'G', 'B']
    img = np.concatenate([channelData[c][...,np.newaxis] for c in colorChannels], axis=2)

    # linear to standard RGB
    img[..., :3] = np.where(img[..., :3] <= 0.0031308,
                                12.92 * img[..., :3],
                                1.055 * np.power(img[..., :3], 1 / 2.4) - 0.055)

    # sanitize image to be in range [0, 1]
    img = np.where(img < 0.0, 0.0, np.where(img > 1.0, 1, img))
    
    return img, depth

path = "../output/images/"
image_prefix = "CLEVR_new_000000"
exrfile = exr.InputFile(path+image_prefix+"camera_ori.exr")
header = exrfile.header()
dw = header['dataWindow']
isize = (dw.max.y - dw.min.y + 1, dw.max.x - dw.min.x + 1)
channelData = dict()
for c in header['channels']:
    C = exrfile.channel(c, Imath.PixelType(Imath.PixelType.FLOAT))
    C = np.frombuffer(C, dtype=np.float32)
    C = np.reshape(C, isize)
        
    channelData[c] = C
    
colorChannels = ['R', 'G', 'B', 'A'] if 'A' in header['channels'] else ['R', 'G', 'B']
img = np.concatenate([channelData[c][...,np.newaxis] for c in colorChannels], axis=2)
    
# linear to standard RGB
img[..., :3] = np.where(img[..., :3] <= 0.0031308,
                            12.92 * img[..., :3],
                            1.055 * np.power(img[..., :3], 1 / 2.4) - 0.055)
    
# sanitize image to be in range [0, 1]
img = np.where(img < 0.0, 0.0, np.where(img > 1.0, 1, img))
img, depth = extract_rgb_depth(path+image_prefix+"camera_ori.exr")


ModuleNotFoundError: No module named 'OpenEXR'

In [11]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(3,2)
fig.set_size_inches(15, 15)
img, depth = extract_rgb_depth(path+image_prefix+"camera1.exr")
ax[0,0].imshow(img)
img, depth = extract_rgb_depth(path+image_prefix+"camera2.exr")
ax[0,1].imshow(img)
img, depth = extract_rgb_depth(path+image_prefix+"camera3.exr")
ax[1,0].imshow(img)
img, depth = extract_rgb_depth(path+image_prefix+"camera4.exr")
ax[1,1].imshow(img)
img, depth = extract_rgb_depth(path+image_prefix+"camera_ori.exr")
ax[2,0].imshow(img)
img, depth = extract_rgb_depth(path+image_prefix+"camera_top.exr")
ax[2,1].imshow(img)

NameError: name 'exr' is not defined

In [1]:
import os
import json
#Here we are opening a path to see the first scene in the json file and 
#we will be comparing it to the RGB image above
with open ('../output/data.json') as f:
    data = json.load(f)
print (list(data['CLEVR_new_000000']['objects']))

['cube_cylinder_01_yellow_metal', 'cube_cylinder_01_purple_metal', 'cylinder_rect_cont_01_large_cyan_metal', 'cube_cylinder_01_large_red_metal', 'short_cylinder_01_purple_metal', 'rectangle_cylinder_01_brown_metal']


In [ ]:
#1. To be able to read the data, how would you open the file? 

In [2]:
# JSON file
f = open ('data.json', "r")
# Reading from file
data = json.loads(f.read())
print(data)

{'CLEVR_new_000000': {'objects': {'cube_cylinder_01_yellow_metal': {'attributes': {'shape': 'cube_cylinder_01', 'color': 'yellow', 'material': 'metal'}, 'relationship': {'cube_cylinder_01': {'can_support': {'cylinder_rect_cont_01_large': True, 'cube_cylinder_01_large': True, 'short_cylinder_01': False, 'rectangle_cylinder_01': True}, 'can_contain': {'cylinder_rect_cont_01_large': False, 'cube_cylinder_01_large': False, 'short_cylinder_01': True, 'rectangle_cylinder_01': False}, 'right_of': {'cube_cylinder_01': False, 'cylinder_rect_cont_01_large': False, 'cube_cylinder_01_large': False, 'short_cylinder_01': True, 'rectangle_cylinder_01': False}, 'left_of': {'cube_cylinder_01': True, 'cylinder_rect_cont_01_large': True, 'cube_cylinder_01_large': True, 'short_cylinder_01': False, 'rectangle_cylinder_01': False}}}}, 'cube_cylinder_01_purple_metal': {'attributes': {'shape': 'cube_cylinder_01', 'color': 'purple', 'material': 'metal'}, 'relationship': {'cube_cylinder_01': {'can_support': {'c

In [7]:
#This is another method
with open("data.json") as file:
    # Load its content and make a new dictionary
    data = json.load(file)
print(data)

{'CLEVR_new_000000': {'objects': {'cube_cylinder_01_yellow_metal': {'attributes': {'shape': 'cube_cylinder_01', 'color': 'yellow', 'material': 'metal'}, 'relationship': {'can_support': {'cube_cylinder_01_purple_metal': True, 'cylinder_rect_cont_01_large_cyan_metal': True, 'cube_cylinder_01_large_red_metal': True, 'short_cylinder_01_purple_metal': False, 'rectangle_cylinder_01_brown_metal': True}, 'can_contain': {'cube_cylinder_01_purple_metal': False, 'cylinder_rect_cont_01_large_cyan_metal': False, 'cube_cylinder_01_large_red_metal': False, 'short_cylinder_01_purple_metal': True, 'rectangle_cylinder_01_brown_metal': False}, 'right_of': {'cube_cylinder_01_purple_metal': False, 'cylinder_rect_cont_01_large_cyan_metal': False, 'cube_cylinder_01_large_red_metal': False, 'short_cylinder_01_purple_metal': False, 'rectangle_cylinder_01_brown_metal': False}, 'left_of': {'cube_cylinder_01_purple_metal': True, 'cylinder_rect_cont_01_large_cyan_metal': True, 'cube_cylinder_01_large_red_metal': T

In [4]:
#2. After extracting the data, how can I find a certain scene?

In [5]:
print(data['CLEVR_new_000002'])

{'objects': {'cube_container_01_yellow_metal': {'attributes': {'shape': 'cube_container_01', 'color': 'yellow', 'material': 'metal'}, 'relationship': {'cube_container_01': {'can_support': {'rectangle_cylinder_01_large': True, 'cylinder_rect_cont_01_large': True, 'long_cylinder_01': True, 'rectangle_cylinder_01': True}, 'can_contain': {'rectangle_cylinder_01_large': False, 'cylinder_rect_cont_01_large': False, 'long_cylinder_01': False, 'rectangle_cylinder_01': False}, 'right_of': {'cube_container_01': False, 'rectangle_cylinder_01_large': False, 'cylinder_rect_cont_01_large': False, 'long_cylinder_01': False, 'rectangle_cylinder_01': False}, 'left_of': {'cube_container_01': True, 'rectangle_cylinder_01_large': True, 'cylinder_rect_cont_01_large': True, 'long_cylinder_01': True, 'rectangle_cylinder_01': False}}}}, 'rectangle_cylinder_01_large_green_rubber': {'attributes': {'shape': 'rectangle_cylinder_01_large', 'color': 'green', 'material': 'rubber'}, 'relationship': {'rectangle_cylind

In [6]:
#3. What are the shape in this the scene? 

In [10]:
print(list(data['CLEVR_new_000002']['objects']))

['cube_container_01_yellow_metal', 'rectangle_cylinder_01_large_green_rubber', 'cylinder_rect_cont_01_large_cyan_rubber', 'long_cylinder_01_cyan_rubber', 'rectangle_cylinder_01_blue_rubber']


In [71]:
#4. For 'long_cylinder_01_yellow_metal' what are their attributes? 

In [11]:
print(data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['attributes'])

{'shape': 'rectangle_cylinder_01', 'color': 'blue', 'material': 'rubber'}


In [12]:
#5.what is the relationship between
#'cube_container_01_purple_metal' and 'cube_container_01' ? 

In [14]:
print(list(data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']))

['can_support', 'can_contain', 'right_of', 'left_of', 'in_front', 'behind']


In [ ]:
#6. How many shapes are in this scene?

In [12]:
print(len(data['CLEVR_new_000002']['objects']))

5


In [ ]:
#7. How many files are in this one file?

In [13]:
print(len(data))

20


In [16]:
#Objects available in the contain relationship: 
print(list(data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['can_contain']))

['cube_container_01_yellow_metal', 'rectangle_cylinder_01_large_green_rubber', 'cylinder_rect_cont_01_large_cyan_rubber', 'long_cylinder_01_cyan_rubber']


In [21]:
#8. Can rectangle_cylinder_01_blue_rubber contain cylinder_rect_cont_01_large_cyan_rubber?
#Works by data['scene'][object]['object_name'][relationship]['shape'][can_contain]['other_shape']

In [18]:
print((data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['can_contain']['cylinder_rect_cont_01_large_cyan_rubber']))
#As seen rectangle_cylinder_01_blue_rubber cannot contain cube_container_01

False


In [20]:
#Objects available in the support relationship:  
print(list(data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['can_support']))

['cube_container_01_yellow_metal', 'rectangle_cylinder_01_large_green_rubber', 'cylinder_rect_cont_01_large_cyan_rubber', 'long_cylinder_01_cyan_rubber']


In [ ]:
#9. In this scene can 'rectangle_cylinder_01_blue_rubber support 'cylinder_rect_cont_01_large_cyan_rubber'
##Works by data['scene'][object]['object_name'][relationship]['shape'][can_support]['other_shape']

In [22]:
print((data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['can_support']['cylinder_rect_cont_01_large_cyan_rubber']))
#As seen rectangle_cylinder_01_blue_rubber can be placed on top of cube_container_01

True


In [24]:
#Objects could be right of the object: 
print(list(data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['right_of']))

['cube_container_01_yellow_metal', 'rectangle_cylinder_01_large_green_rubber', 'cylinder_rect_cont_01_large_cyan_rubber', 'long_cylinder_01_cyan_rubber']


In [27]:
#Is 'rectangle_cylinder_01_blue_rubber' right of 'cylinder_rect_cont_01_large_cyan_rubber'? 
print((data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['right_of']['cylinder_rect_cont_01_large_cyan_rubber']))
#It Should Not

False


In [28]:
#Is 'rectangle_cylinder_01_blue_rubber' in front of 'cylinder_rect_cont_01_large_cyan_rubber'? 
print((data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['in_front']['cylinder_rect_cont_01_large_cyan_rubber']))

True


In [29]:
#Objects could be behind of the object: 
#Is 'rectangle_cylinder_01_blue_rubber' in behind 'cylinder_rect_cont_01_large_cyan_rubber'? 
print(data['CLEVR_new_000002']['objects']['rectangle_cylinder_01_blue_rubber']['relationship']['behind']['cylinder_rect_cont_01_large_cyan_rubber'])

False
